# kerasに慣れよう1
入力ベクトル$(x_{1},x_{2},x_{3},x_{4},x_{5})$ ($x_i \in [0,1]$)の各要素の加算結果が2.5以上で1,未満で0を出すモデル  
$f(x) = if\ \sum^5_{i=1}x_i \ \geq\ 2.5 \ then\  1\  else\  0$  
参考:[無から始めるKeras 第1回][1]
[1]:https://qiita.com/Ishotihadus/items/c2f864c0cde3d17b7efb

## ライブラリなどの準備

In [1]:
#ライブラリ
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation

#GPUの仕様に関する設定
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(
        visible_device_list = "0",
        allow_growth = True,
        per_process_gpu_memory_fraction = 0.3))
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


## データセットの生成

In [2]:
#学習データセット
data = np.random.rand(10000,5) #1000個の5次元ベクトル
labels = (np.sum(data, axis=1) > 2.5) * 1 #ラベル(0,1)
labels = np_utils.to_categorical(labels)  #ラベル(onehot)
#onehotラベルはこんな感じ
#学習データ
print("学習データ")
print(data[:3])
print("正解(onehot)")
print(labels[:3])

学習データ
[[0.63824372 0.51774281 0.11788814 0.8232685  0.94268798]
 [0.2111682  0.51023754 0.48468407 0.60537811 0.15774433]
 [0.13077407 0.65198469 0.33986234 0.69184605 0.57739458]]
正解(onehot)
[[0. 1.]
 [1. 0.]
 [1. 0.]]


## ネットワークの構築

In [3]:
#Sequential:層を積み上げる単純なモデル
model = Sequential()
model.add(Dense(20, input_dim=5))
model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

#こんな書き方もできる
# model = Sequential([
#     Dense(20, input_dim=5),
#     Activation('relu'),
#     Dense(2,activation='softmax')
# ])

#ネットワーク構造の出力
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                120       
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 42        
Total params: 162
Trainable params: 162
Non-trainable params: 0
_________________________________________________________________


## 学習

In [4]:
#compli:最適化関数,損失関数,評価指標
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
#学習
#fit;入力，出力，バッチサイズ(一度の更新に使用するデータ数), エポック数(学習回数),検証セットの割合(ランダムではないので注意)
model.fit(data, labels, batch_size = 100, epochs=150, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 1s 90us/step - loss: 0.6726 - acc: 0.5425 - val_loss: 0.6510 - val_acc: 0.6005
Epoch 2/150
8000/8000 [==============================] - 0s 41us/step - loss: 0.6219 - acc: 0.6561 - val_loss: 0.5908 - val_acc: 0.7110
Epoch 3/150
8000/8000 [==============================] - 0s 41us/step - loss: 0.5509 - acc: 0.7800 - val_loss: 0.5123 - val_acc: 0.8300
Epoch 4/150
8000/8000 [==============================] - 0s 41us/step - loss: 0.4668 - acc: 0.8726 - val_loss: 0.4295 - val_acc: 0.8965
Epoch 5/150
8000/8000 [==============================] - 0s 41us/step - loss: 0.3872 - acc: 0.9237 - val_loss: 0.3578 - val_acc: 0.9465
Epoch 6/150
8000/8000 [==============================] - 0s 42us/step - loss: 0.3224 - acc: 0.9616 - val_loss: 0.3024 - val_acc: 0.9625
Epoch 7/150
8000/8000 [==============================] - 0s 42us/step - loss: 0.2740 - acc: 0.9750 - val_loss: 0.2617 - val_acc: 0.9820


Epoch 120/150
8000/8000 [==============================] - 0s 44us/step - loss: 0.0314 - acc: 0.9961 - val_loss: 0.0341 - val_acc: 0.9960
Epoch 121/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0312 - acc: 0.9959 - val_loss: 0.0337 - val_acc: 0.9955
Epoch 122/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0314 - acc: 0.9941 - val_loss: 0.0342 - val_acc: 0.9945
Epoch 123/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0308 - acc: 0.9959 - val_loss: 0.0334 - val_acc: 0.9955
Epoch 124/150
8000/8000 [==============================] - 0s 42us/step - loss: 0.0307 - acc: 0.9965 - val_loss: 0.0332 - val_acc: 0.9955
Epoch 125/150
8000/8000 [==============================] - 0s 43us/step - loss: 0.0305 - acc: 0.9963 - val_loss: 0.0330 - val_acc: 0.9950
Epoch 126/150
8000/8000 [==============================] - 0s 42us/step - loss: 0.0304 - acc: 0.9963 - val_loss: 0.0330 - val_acc: 0.9965
Epoch 127/150
8000/8000 [=========

## 未知データの予測

In [5]:
#評価データセット
test_data = np.random.rand(1000,5)
test_label = (np.sum(test_data, axis=1) > 2.5) * 1 

In [6]:
print("入力")
print(test_data[0])
print("正解")
print(test_label[0])
print("予測結果の確率(モデルの出力)")
#出力結果の確率
print(model.predict(test_data[0:1]))
print("確率が最大の次元")
#最も大きい次元を出力とする
print(np.argmax(model.predict(test_data[0:1])))

入力
[0.42922686 0.10192073 0.70005548 0.57467436 0.59286714]
正解
0
予測結果の確率(モデルの出力)
[[0.98598576 0.01401428]]
確率が最大の次元
0


In [7]:
predict = np.argmax(model.predict(test_data), axis=1)
#predictの中身
print("各データに対する予測")
print(predict[:3])
print("正解と予測の比較")
print((predict==test_label)[:3])
#sumを取るとtrueの数を集計する
print("認識率")
print(sum(predict == test_label) /1000)

各データに対する予測
[0 0 1]
正解と予測の比較
[ True  True  True]
認識率
0.996


## モデルの中身(重みとバイアスを取得)

In [8]:
weight = model.get_weights()
print(weight[0]) #隠れ層重み
print(weight[1]) #隠れ層バイアス
print(weight[2]) #出力層重み
print(weight[3]) #出力層バイアス

[[ 0.07245422 -0.76324856  0.7859789  -0.31263816  0.28501177 -0.52260983
  -0.5815298   0.87844     0.8093793  -0.22135888 -0.2908083   0.7141679
   0.73738694 -0.4658815  -0.36737716 -0.10367944 -0.38862735  0.35797924
   0.6213824   0.86724794]
 [-0.5889794  -0.20837772  0.5896446  -0.44063967 -0.02502468 -0.12219965
   0.0246519   0.70582134  1.0986743   0.05536262 -0.79370195  0.61077756
   0.72866404  0.17121376 -0.4394324   0.06184224 -0.3990213   0.9721677
   0.59114563  1.0119864 ]
 [-0.4692893  -0.3926169   0.78621554 -0.06523511 -0.6226341  -0.4768676
  -0.9288128   0.5054797   0.3359503  -0.37780178 -0.24653074  0.89912605
   0.86226207 -0.1142491   0.2296202  -0.58266085 -0.21870618  0.6524426
   0.7657701   0.2795029 ]
 [-0.4653854  -0.436262    0.70475334 -0.11117026  0.35039404 -0.36173663
   0.145712    0.6153372   0.85476017  0.0274701  -0.1293269   0.5208152
   0.8943596  -0.1378539  -0.48768464 -0.69694525 -0.5728259   0.41966832
   0.71036476  0.9184199 ]
 [-0.6655